In [1]:
!rmdir /S /Q lightning_logs

In [2]:
import feature_engine.imputation
import feature_engine.pipeline
import lightning
import pandas
import torch
import torch.nn
import torchmetrics

In [3]:
lightning.seed_everything(0)

Seed set to 0


0

In [4]:
train_param_df = pandas.read_csv("train.csv", header=None)
train_param_df.columns =  [f'column_{i}' for i in range(train_param_df.shape[1])]
train_param_df

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,0.097094,1.11330,45.038,0.88184,0.087009,1.04100,1.548600,3.49800,1.85780,0.009673,...,0.076209,3.66540,0.061607,0.003161,0.036038,0.084500,2.45170,3.3373,0.065201,0.091158
1,0.050086,0.11158,94.080,1.76500,0.089417,4.80470,0.267420,NaN,0.56473,0.035123,...,0.054712,4.16870,0.075432,0.010869,0.063972,0.079892,1.97950,3.5064,0.072132,0.091950
2,0.088447,2.36340,5.058,0.14436,0.064547,2.44400,4.254500,0.36506,1.86090,0.009759,...,0.017203,4.56130,0.046505,NaN,0.084066,0.064829,3.30870,2.9969,0.064328,0.036793
3,0.772540,0.59469,NaN,0.97515,0.015987,0.52884,1.488400,3.96100,4.80630,0.048617,...,0.022891,0.12832,0.065028,0.036862,0.010010,0.020709,2.52370,2.1711,0.080865,0.081553
4,0.382410,4.81090,1955.100,0.46050,0.024453,2.02980,3.740300,4.22810,2.42920,0.156830,...,0.032051,4.37010,1.001100,0.065750,0.043547,0.629430,4.62620,3.1947,NaN,0.187180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.083012,4.44000,1140.100,5.04650,0.069693,4.32170,2.652700,4.80340,4.91870,0.088879,...,0.054703,3.50690,0.518980,0.047767,0.061994,0.256460,3.57210,4.7692,0.056299,0.162660
596,0.523780,3.32790,122.050,3.54630,1.010900,1.72230,3.131200,3.75900,3.61270,0.061250,...,0.351960,1.55980,0.023365,1.084000,0.072553,0.049359,3.46930,3.1465,0.031458,0.019107
597,0.995510,1.60620,464.090,3.58910,0.091039,1.62010,3.688400,1.84550,0.41606,0.548030,...,0.114580,3.32090,0.019906,0.057709,0.097567,0.073208,0.62583,4.6044,0.071155,0.053476
598,1.208000,2.10000,24.011,2.92140,0.055663,0.78477,1.288900,3.18080,4.77850,0.050453,...,0.011022,2.48810,0.061171,0.070312,NaN,0.043862,0.58203,4.5994,0.045182,0.031991


In [5]:
pipeline = feature_engine.pipeline.make_pipeline(
    feature_engine.imputation.MeanMedianImputer(imputation_method='median'),
)

In [6]:
train_param_processed_df = pipeline.fit_transform(train_param_df)
train_param_processed_df

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,0.097094,1.11330,45.0380,0.88184,0.087009,1.04100,1.548600,3.49800,1.85780,0.009673,...,0.076209,3.66540,0.061607,0.003161,0.036038,0.084500,2.45170,3.3373,0.065201,0.091158
1,0.050086,0.11158,94.0800,1.76500,0.089417,4.80470,0.267420,2.50960,0.56473,0.035123,...,0.054712,4.16870,0.075432,0.010869,0.063972,0.079892,1.97950,3.5064,0.072132,0.091950
2,0.088447,2.36340,5.0580,0.14436,0.064547,2.44400,4.254500,0.36506,1.86090,0.009759,...,0.017203,4.56130,0.046505,0.051281,0.084066,0.064829,3.30870,2.9969,0.064328,0.036793
3,0.772540,0.59469,93.0575,0.97515,0.015987,0.52884,1.488400,3.96100,4.80630,0.048617,...,0.022891,0.12832,0.065028,0.036862,0.010010,0.020709,2.52370,2.1711,0.080865,0.081553
4,0.382410,4.81090,1955.1000,0.46050,0.024453,2.02980,3.740300,4.22810,2.42920,0.156830,...,0.032051,4.37010,1.001100,0.065750,0.043547,0.629430,4.62620,3.1947,0.057666,0.187180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.083012,4.44000,1140.1000,5.04650,0.069693,4.32170,2.652700,4.80340,4.91870,0.088879,...,0.054703,3.50690,0.518980,0.047767,0.061994,0.256460,3.57210,4.7692,0.056299,0.162660
596,0.523780,3.32790,122.0500,3.54630,1.010900,1.72230,3.131200,3.75900,3.61270,0.061250,...,0.351960,1.55980,0.023365,1.084000,0.072553,0.049359,3.46930,3.1465,0.031458,0.019107
597,0.995510,1.60620,464.0900,3.58910,0.091039,1.62010,3.688400,1.84550,0.41606,0.548030,...,0.114580,3.32090,0.019906,0.057709,0.097567,0.073208,0.62583,4.6044,0.071155,0.053476
598,1.208000,2.10000,24.0110,2.92140,0.055663,0.78477,1.288900,3.18080,4.77850,0.050453,...,0.011022,2.48810,0.061171,0.070312,0.056169,0.043862,0.58203,4.5994,0.045182,0.031991


In [7]:
train_label_df = pandas.read_csv("train_labels.csv", header=None)
train_label_df.columns = ["target"]
train_label_df

,target
0,0
1,1
2,0
3,0
4,1
...,...
595,1
596,0
597,1
598,0


In [8]:
train_label_df["target"].values

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,

In [9]:
test_df = pandas.read_csv("test.csv", header=None)
test_df.columns =  [f'column_{i}' for i in range(test_df.shape[1])]
test_df

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,0.131200,4.20140,1428.1000,2.90940,NaN,2.66500,4.54560,0.62337,4.76250,4.287300,...,0.099056,2.0518,0.008945,0.087689,0.070505,0.029144,1.16970,0.50162,0.045636,0.142370
1,0.083796,3.92690,80.0590,1.87800,0.064623,4.14680,3.72740,0.10988,1.90250,0.093267,...,0.046586,2.9683,0.079120,0.079630,0.026689,0.066626,1.94460,2.21630,0.099828,0.088621
2,0.067591,3.01070,25.0300,1.26430,3.031100,3.09690,1.78210,1.43460,1.88170,0.002060,...,0.099853,1.2933,0.015907,0.061634,0.079210,0.022622,2.01960,3.44300,0.069516,0.025426
3,0.064830,0.11960,5.0957,1.93660,0.028638,1.21470,0.34675,1.11880,0.99529,0.060067,...,0.095575,4.4767,NaN,0.067340,0.054280,0.003331,0.95697,3.69700,0.049861,0.073140
4,0.345720,1.42610,44.0410,2.76560,0.095754,1.70890,3.94490,0.11820,1.50260,0.030124,...,0.210120,1.2719,0.028321,0.086113,0.007885,0.528200,1.87460,2.14880,0.030729,0.020493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.036445,3.86630,530.0100,3.19080,0.043165,0.21049,1.61840,2.67970,4.23950,0.075493,...,0.177070,3.7940,0.021300,0.082044,0.030834,0.068292,3.56710,2.97000,0.093883,0.056370
3996,0.080715,3.06090,145.0900,1.74360,0.051496,4.04320,4.01490,3.08130,2.81060,0.022633,...,0.029050,1.5209,0.023879,0.084377,0.005575,0.119900,1.37370,1.35300,0.068311,NaN
3997,0.260410,4.01400,1161.1000,0.58602,0.826840,0.31077,1.59060,3.87060,3.56340,0.090322,...,0.226540,4.0418,0.070592,0.014543,0.014508,0.018798,1.90200,4.91760,0.115350,0.018841
3998,0.048965,0.75552,474.0100,2.68170,0.050257,2.62300,4.58700,1.96530,1.20000,0.004702,...,0.033444,NaN,0.090492,0.013366,0.030227,0.063834,3.21960,3.78840,0.016478,0.449020


In [10]:
train_df = train_param_df.join(train_label_df)
train_df

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99,target
0,0.097094,1.11330,45.038,0.88184,0.087009,1.04100,1.548600,3.49800,1.85780,0.009673,...,3.66540,0.061607,0.003161,0.036038,0.084500,2.45170,3.3373,0.065201,0.091158,0
1,0.050086,0.11158,94.080,1.76500,0.089417,4.80470,0.267420,NaN,0.56473,0.035123,...,4.16870,0.075432,0.010869,0.063972,0.079892,1.97950,3.5064,0.072132,0.091950,1
2,0.088447,2.36340,5.058,0.14436,0.064547,2.44400,4.254500,0.36506,1.86090,0.009759,...,4.56130,0.046505,NaN,0.084066,0.064829,3.30870,2.9969,0.064328,0.036793,0
3,0.772540,0.59469,NaN,0.97515,0.015987,0.52884,1.488400,3.96100,4.80630,0.048617,...,0.12832,0.065028,0.036862,0.010010,0.020709,2.52370,2.1711,0.080865,0.081553,0
4,0.382410,4.81090,1955.100,0.46050,0.024453,2.02980,3.740300,4.22810,2.42920,0.156830,...,4.37010,1.001100,0.065750,0.043547,0.629430,4.62620,3.1947,NaN,0.187180,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.083012,4.44000,1140.100,5.04650,0.069693,4.32170,2.652700,4.80340,4.91870,0.088879,...,3.50690,0.518980,0.047767,0.061994,0.256460,3.57210,4.7692,0.056299,0.162660,1
596,0.523780,3.32790,122.050,3.54630,1.010900,1.72230,3.131200,3.75900,3.61270,0.061250,...,1.55980,0.023365,1.084000,0.072553,0.049359,3.46930,3.1465,0.031458,0.019107,0
597,0.995510,1.60620,464.090,3.58910,0.091039,1.62010,3.688400,1.84550,0.41606,0.548030,...,3.32090,0.019906,0.057709,0.097567,0.073208,0.62583,4.6044,0.071155,0.053476,1
598,1.208000,2.10000,24.011,2.92140,0.055663,0.78477,1.288900,3.18080,4.77850,0.050453,...,2.48810,0.061171,0.070312,NaN,0.043862,0.58203,4.5994,0.045182,0.031991,0


In [11]:
class MyModel(lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._stack = torch.nn.Sequential(
            torch.nn.Linear(100, 200), torch.nn.ReLU(),
            torch.nn.Linear(200, 100), torch.nn.ReLU(),
            torch.nn.Linear(100, 2), torch.nn.Softmax(dim=1),
        )
        self._accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)

    def forward(self, x):
        return self._stack(x)
        
    def training_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        return loss

    def test_step(self, batch, batch_idx):
        loss, acc = self._compute(batch, batch_idx)
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    
    def _compute(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        acc = self._accuracy(y_hat, y)
        return (loss, acc)

In [12]:
class MyLoader(lightning.LightningDataModule):
    def __init__(self):
        super().__init__()
        train_param = torch.tensor(train_param_processed_df.values, dtype=torch.float32)
        train_label = torch.tensor(train_label_df["target"].values, dtype=torch.int64)
        dataset = torch.utils.data.TensorDataset(train_param, train_label)
        n_train = int(len(train_df) * 0.8)
        n_val = int(len(train_df) * 0.1)
        n_test = len(train_df) - n_train - n_val
        self._train_set, self._val_set, self._test_set = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self._train_set)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self._val_set)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self._test_set)

In [13]:
trainer = lightning.Trainer(callbacks=[lightning.pytorch.callbacks.early_stopping.EarlyStopping(monitor="val_loss", mode="min")], max_epochs=10)
model = MyModel()
loader = MyLoader()
trainer.fit(model, loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | _stack    | Sequential         | 40.5 K | train
1 | _accuracy | MulticlassAccuracy | 0      | train
---------------------------------------------------------
40.5 K    Trainable params
0         Non-trainable params
40.5 K    Total params
0.162     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

In [14]:
trainer.test(datamodule=loader)

C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at C:\Users\hsmtk\repo\fantastic-octo-dollop\just-the-basics-the-after-party\lightning_logs\version_0\checkpoints\epoch=3-step=1920.ckpt
Loaded model weights from the checkpoint at C:\Users\hsmtk\repo\fantastic-octo-dollop\just-the-basics-the-after-party\lightning_logs\version_0\checkpoints\epoch=3-step=1920.ckpt
C:\Users\hsmtk\repo\fantastic-octo-dollop\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the

Testing: |                                                                                       | 0/? [00:00<…

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.5666666626930237
        test_loss           0.7465952634811401
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.7465952634811401, 'test_acc': 0.5666666626930237}]

In [15]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 19440), started 11:36:57 ago. (Use '!kill 19440' to kill it.)